In [ ]:
# Reload our package (for use if modified).
import importlib
importlib.reload(load)
importlib.reload(normalize)
importlib.reload(process)
importlib.reload(batch)
importlib.reload(cnn)
importlib.reload(metrics)

In [1]:
# Put the project's root directory on our PYTHONPATH.
import sys
sys.path.insert(0, "../..")

In [2]:
# Import from our package.
from src.preprocessing import load, normalize, process, batch
from src.classification import cnn, metrics

In [3]:
# Import from other libraries.
import tensorflow as tf
import random
import numpy as np

In [4]:
# Define the architecture of our net as a list of layers.
layers = [
    cnn.ConvolutionalLayer(5, 5, 1, "VALID", "convlayer"),
    cnn.TransferLayer(lambda x, name: tf.sigmoid(x, name=name), "trans1layer"),
    cnn.ConvolutionalLayer(20, 3, 1, "VALID", "conv2layer"),
    cnn.TransferLayer(lambda x, name: tf.sigmoid(x, name=name), "trans2layer"),
    cnn.FlatLayer("flatlayer"),
    cnn.FullLayer(100, "full1layer"),
    cnn.TransferLayer(lambda x, name: tf.sigmoid(x, name=name), "trans3layer"),
    cnn.FullLayer(100, "full2layer"),
    cnn.TransferLayer(lambda x, name: tf.sigmoid(x, name=name), "trans4layer"),
    cnn.FullLayer(2, "full3layer")
]

In [ ]:
# Construct the net.
sess = tf.Session()
net = cnn.NeuralNet(
    layers,
    lambda labels, predictions: tf.losses.softmax_cross_entropy(
        labels,
        predictions,
        weights=1,
        label_smoothing=0,
        scope=None,
        loss_collection=tf.GraphKeys.LOSSES,
        reduction=tf.losses.Reduction.SUM_BY_NONZERO_WEIGHTS
    ),
    {
        "accuracy": metrics.accuracy,
        "false_negative_rate": metrics.false_negative_rate,
        "false_positive_rate": metrics.false_positive_rate,
        "true_positive_rate": metrics.true_positive_rate,
        "true_negative_rate": metrics.true_negative_rate,
        "roc_auc": metrics.roc_auc
    },
    lambda: tf.train.MomentumOptimizer(0.01, 0.5),
    "models/" + "14",
    "nn"  
)

INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_secs': 600, '_session_config': None, '_tf_random_seed': 1, '_save_summary_steps': 100, '_model_dir': 'models/13'}


In [ ]:
# Load and preprocess training data.
train_images, train_masks = load.load_images_and_masks("data/train", [str(i) for i in range(900)])
train_images = normalize.smdm_normalize(train_images, 21, "REFLECT")
train_patches, train_classes = process.patchify(train_images, train_masks, 21, 4)
train_classes = process.one_hot_encode(train_classes)
train_input_fn = batch.input_fn(train_patches, train_classes, 5000)

In [ ]:
# Train the net for 5 minutes.
with tf.Session().as_default():
    net.train_duration_(train_input_fn, 300)

In [ ]:
# Load and preprocess test data.
test_images, test_masks = load.load_images_and_masks("data/test", [str(i) for i in range(100)])
test_images = normalize.smdm_normalize(test_images, 21, "REFLECT")
test_patches, test_classes = process.patchify(test_images, test_masks, 21, 4)
test_classes = process.one_hot_encode(test_classes)
test_input_fn = batch.input_fn(test_patches, test_classes, 5000, 1)

In [ ]:
# Evaluate the net.
metric_dict = net.evaluate_(test_input_fn)

In [ ]:
# Examine the test results.
metric_dict